to know how many cases are in this file

In [47]:
import fitz
import re

def extract_target_number(pdf_path):
    doc = fitz.open(pdf_path)

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if (
                            'number' in b and b['number'] == 5
                            and 'size' in span and span['size'] == 12.0
                            and 'font' in span and span['font'] == 'Helvetica-Bold'
                            and page_num == 0  # Only on the first page
                        ):
                            match = re.search(r'\((\d+)\)', span["text"])
                            if match:
                                return int(match.group(1))

    doc.close()


get the title list

In [52]:
import fitz
import re

def extract_title(pdf_path):
    doc = fitz.open(pdf_path)
    extracted_text_list = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if 'font' in span and span['font'] == 'Helvetica-Oblique':
                            result_string = re.sub(r', \d+ [A-Za-z. ]+ \d+$', '', span["text"])
                            extracted_text_list.append(result_string.strip())
        if len(extracted_text_list) >= extract_target_number(pdf_path):
            break

    doc.close()
    return extracted_text_list

In [53]:
pdf_file_path = "D:/antitrust/verdict/Files (100).PDF"
# pdf_file_path = "D:/antitrust/verdict/Federal District (30).PDF"
result_list = extract_title(pdf_file_path)

In [54]:
result_list

['United States v. International Boxing Club',
 'INTERNATIONAL BOXING CLUB OF NEW YORK, INC. v. UNITED STATES',
 'UNITED STATES v. RCA',
 'Minneapolis & S. L. R. Co. v. United States',
 'United States v. Parke',
 "Maryland & Virginia Milk Producers Ass'n v. United States",
 'International Brotherhood of Teamsters, etc. v. Oliver',
 'United States v. E. I. du Pont de Nemours & Co.',
 "California v. Federal Power Comm'n",
 "United States v. Loew's",
 'White Motor Co. v. United States',
 "United States v. Philadelphia Nat'l Bank",
 "United States v. First Nat'l Bank & Trust Co.",
 'United States v. Continental Can Co.',
 'United States v. Boston & M. R.R.',
 'Columbia Artists Management v. United States',
 'Minnesota Mining & Mfg. Co. v. New Jersey Wood Finishing Co.',
 'FTC v. Borden Co.',
 'United States v. General Motors Corp.',
 "United States v. Von's Grocery Co.",
 'FTC v. Brown Shoe Co.',
 'FTC v. Dean Foods Co.',
 'Denver & R. G. W. R. Co. v. United States',
 'Kaplan v. Lehman Bro

In [55]:
import csv

get other information and write them into csv

In [56]:
import fitz

def get_texts_matching_cases(pdf_path, case_list, output_csv):
    doc = fitz.open(pdf_path)
    matching_texts = []
    title = None  # Variable to store 'number': 1 text
    court = None  # Variable to store 'number': 2 text
    date = None  # Variable to store 'number': 3 text
    no = None  # Variable to store 'number': 4 text
    consider_number_2 = False  # Flag to indicate whether to consider 'number': 2 text
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if 'number' in b and b['number'] == 1:
                if "lines" in b:
                    # Reset text variables for each new 'number': 1
                    title = None
                    court = None
                    date = None
                    no = None

                    for line in b["lines"]:
                        for span in line["spans"]:
                            if 'font' in span and span['font'] == 'Helvetica-BoldOblique':
                                for case_name in case_list:
                                    if span["text"].strip() == case_name.strip():
                                        title = span["text"].strip()
                                        # Set the flag to consider 'number': 2 and 'number': 3 text
                                        consider_number_2 = True
                                        break  # Stop looking for 'number': 1 text once found
            if consider_number_2 and 'number' in b and b['number'] == 2:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                court = span["text"].strip()
            if consider_number_2 and 'number' in b and b['number'] == 3:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                date = span["text"].strip()
            if consider_number_2 and 'number' in b and b['number'] == 4:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                no = span["text"].strip()
                                # Stop considering 'number': 2, 'number': 3, and 'number': 4 text
                                consider_number_2 = False
                                break  # Stop looking for 'number': 4 text once found
        # Check if 'number': 1, 'number': 2, 'number': 3, and 'number': 4 text are present, then append to the result
        if no:
            matching_texts.append({'Title': title, 'Court': court, 'Date': date, 'No': no, 'page':page_num })
    
    doc.close()
    # Save matching_texts to a CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['Title', 'Court', 'Date', 'No', 'page']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # Write the header
        writer.writeheader()
        
        # Write the data
        for result in matching_texts:
            writer.writerow(result)




In [57]:
result_texts = get_texts_matching_cases(pdf_file_path, result_list, 'index_output.csv')

In [58]:
import fitz

def get_texts_matching_cases(pdf_path, case_list, output_csv):
    doc = fitz.open(pdf_path)
    matching_texts = []
    text_variables = [None] * 4  # Initialize text variables to None
    
    consider_number_2 = False  # Flag to indicate whether to consider 'number': 2 text
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if 'number' in b and b['number'] == 1:
                if "lines" in b:
                    # Reset text variables for each new 'number': 1
                    text_variables = [None] * 4

                    for line in b["lines"]:
                        for span in line["spans"]:
                            if 'font' in span and span['font'] == 'Helvetica-BoldOblique':
                                for i, case_name in enumerate(case_list):
                                    if span["text"].strip() == case_name.strip():
                                        text_variables[0] = span["text"].strip()
                                        # Set the flag to consider 'number': 2, 'number': 3, and 'number': 4 text
                                        consider_number_2 = True
                                        break  # Stop looking for 'number': 1 text once found
            elif consider_number_2 and 'number' in b and 2 <= b['number'] <= 4:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                text_variables[b['number'] - 1] = span["text"].strip()
                
                if b['number'] == 4:
                    # Stop considering 'number': 2, 'number': 3, and 'number': 4 text
                    consider_number_2 = False
                    # Check if 'number': 1, 'number': 2, 'number': 3, and 'number': 4 text are present, then append to the result
                    if text_variables[3]:
                        matching_texts.append({
                            'Title': text_variables[0],
                            'Court': text_variables[1],
                            'Date': text_variables[2],
                            'No': text_variables[3],
                            'page': page_num
                        })
    
    doc.close()
    
    # Save matching_texts to a CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['Title', 'Court', 'Date', 'No', 'page']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # Write the header
        writer.writeheader()
        
        # Write the data
        for result in matching_texts:
            writer.writerow(result)




In [59]:
get_texts_matching_cases(pdf_file_path, result_list, 'output.csv')